In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Adding dispersion to the CHGNet pre-trained model

This notebook describes the process of adding a dispersion correction to the CHGNet pre-trained model. CHGNet is trained on PBE (GGA) DFT calculations; as such, it does not include a correction for van der Waals or dispersive forces. This kind of correction may be particularly useful for those studying porous materials, such as MOFs or zeolites, but who do not wish to fine-tune the pre-trained model on data that include a dispersion correction.

This notebook uses both the [torch-dftd](https://github.com/pfnet-research/torch-dftd/tree/master) and [DFT-D4](https://dftd4.readthedocs.io/en/latest/reference/ase.html) repositories to add dispersion to CHGNet. The torch-dftd repository currently has DFT-D2 and DFT-D3 implementations and does not have the most recent DFT-D4 version, but is GPU-accelerated where DFT-D4 is not. The Grimme group has released a version of [DFT-D4 implemented in PyTorch](https://github.com/dftd4/tad-dftd4); however, this version does not have an ASE-compatible calculator available.

You will need to install CHGNet, [ASE](https://wiki.fysik.dtu.dk/ase/install.html), [torch-dftd](https://github.com/pfnet-research/torch-dftd/tree/master?tab=readme-ov-file#install), and [DFT-D4](https://dftd4.readthedocs.io/en/latest/recipe/installation.html) to run this notebook (links are to their installation instructions).

In [ ]:
from ase.build import fcc111
from ase.calculators.mixing import SumCalculator
from dftd4.ase import DFTD4
from torch_dftd.torch_dftd3_calculator import TorchDFTD3Calculator

from chgnet.model.dynamics import CHGNetCalculator

In [ ]:
# pre-trained chgnet model
chgnet_calc = CHGNetCalculator()

d3_calc = TorchDFTD3Calculator()  # uses PBE parameters by default

d4_calc = DFTD4(method="PBE")

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
CHGNet will run on cpu


## A simple example

This example shows how to initialize an Atoms object (of a Cu(111) surface) and compute its energy with and without the dispersion correction.

In [ ]:
# Create a 2x2x1 fcc(111) Cu slab
atoms = fcc111("Cu", (2, 2, 1), vacuum=10.0)
atoms.set_pbc([True, True, True])

atoms_disp = atoms.copy()
atoms_d4 = atoms.copy()

atoms.calc = chgnet_calc

chgnet_d3 = SumCalculator([chgnet_calc, d3_calc])
chgnet_d4 = SumCalculator([chgnet_calc, d4_calc])
atoms_disp.calc = chgnet_d3
atoms_d4.calc = chgnet_d4

e_chg = atoms.get_potential_energy()
e_disp = atoms_disp.get_potential_energy()
e_d4 = atoms_d4.get_potential_energy()

print(f"Disp calculator: {chgnet_d3.name}")
print(atoms.get_chemical_formula())
print(f"E without dispersion: {e_chg}")
print(f"E with DFT-D3 dispersion: {e_disp}")
print(f"E with DFT-D4 dispersion: {e_d4}")

Disp calculator: sumcalculator
Cu4
E without dispersion: -12.876540184020996
E with DFT-D3 dispersion: -13.272150007989707
E with DFT-D4 dispersion: -13.573149493981


## Optimization example

Below is a simple example of an optimization of a Cu cell with a displaced atom and perturbed unit cell.

In [ ]:
from ase.build import bulk
from ase.filters import FrechetCellFilter
from ase.optimize import BFGS

atoms = bulk("Cu", cubic=True)

atoms[0].x += 0.1
atoms.cell[0] += 0.1

In [ ]:
atoms.cell

Cell([[3.71, 0.1, 0.1], [0.0, 3.61, 0.0], [0.0, 0.0, 3.61]])

In [ ]:
atoms.calc = chgnet_d4
cell_filter = FrechetCellFilter(atoms)
opt = BFGS(cell_filter, trajectory="Cu.traj")

opt.run(fmax=0.5, steps=100)

      Step     Time          Energy          fmax
BFGS:    0 17:55:31      -18.448409        0.718576
BFGS:    1 17:55:33      -18.479250        0.649700
BFGS:    2 17:55:34      -18.602240     1352.772997
BFGS:    3 17:55:35      -18.495526        0.666888
BFGS:    4 17:55:36      -18.505485        0.674898
BFGS:    5 17:55:37      -18.524050        0.707074
BFGS:    6 17:55:39      -18.524685        0.708438
BFGS:    7 17:55:40      -18.527744        0.722830
BFGS:    8 17:55:41      -18.529436        0.740565
BFGS:    9 17:55:42      -18.530648        0.755404
BFGS:   10 17:55:43      -18.530939        0.757174
BFGS:   11 17:55:44      -18.531477        0.756699
BFGS:   12 17:55:46      -18.532681        0.750869
BFGS:   13 17:55:47      -18.534685        0.741311
BFGS:   14 17:55:48      -18.537005        0.728779
BFGS:   15 17:55:49      -18.539452        0.706232
BFGS:   16 17:55:50      -18.540848        0.684654
BFGS:   17 17:55:52      -18.541594        0.680409
BFGS:   18 17:

True

The output of this optimization can be viewed by running

```bash
ase gui Cu.traj
```

in the command line in this folder.